# 向量加法

在本教程中，您将使用 Triton 编写一个简单的向量加法。

在此过程中，您将了解：

* Triton 的基本编程模型。

* 用于定义 Triton 内核的 `triton.jit` 装饰器。

* 验证和基准测试自定​​义操作与原生参考实现的最佳实践。

## 计算内核（Compute Kernel）



我们先导入需要的库，并获取当前可用的 GPU 设备：

In [ ]:
import torch

import triton
import triton.language as tl

DEVICE = triton.runtime.driver.active.get_active_torch_device()

Triton 的 Kernel 本质上就是一个函数，但和普通 Python 函数不同，它不会在 CPU 上执行，而是会被编译成 GPU 上运行的并行代码。
在这里，我们要写一个 Kernel 来实现“两个向量相加”的操作。

在 GPU 上，逐元素的 “x+y” 并不是一次性在整个向量上完成的，而是被拆成：
1. 从显存把一段数据装入寄存器；
2. 在寄存器里做逐元素加法；
3. 把结果写回显存。
Triton 的 kernel 就是对这三步的显式描述，并允许我们决定“数据如何被分片并行”。

下面是 Kernel 的定义。它接收三个向量的指针：`x_ptr`、`y_ptr` 和 `output_ptr`，分别对应输入和输出。参数 `n_elements` 表示向量的总长度，而 `BLOCK_SIZE` 是一个编译期常量，用来决定每个 GPU 程序（program）一次要处理多少个元素。

In [17]:
@triton.jit
def add_kernel(
    x_ptr,        # 输入向量 x 的显存指针（指向 device/global memory 的起始地址）
    y_ptr,        # 输入向量 y 的显存指针
    output_ptr,   # 输出向量的显存指针；可与 x_ptr 相同实现就地写回，但通常单独更清晰
    n_elements,   # 运行时的元素总数（不在编译期固定）
    BLOCK_SIZE: tl.constexpr,  # 编译期常量：每个“并行实例”（program）处理的元素数
):
    # —— grid / program / program_id（并行实例与编号）———————————————
    # 一次 kernel 启动会并行地创建很多“program”（可理解为同一个内核代码的多个实例）。
    # 这些实例排列在一个网格（grid）上。grid 可以是一维、二维或三维：
    #   • 一维 grid：只在一个维度上编号，适合处理一条长向量或一维分片。
    #   • 二维 grid：在两个维度上编号（例如 M×N 的 tile 网格），常用于矩阵乘/卷积等把数据
    #     同时按“行块×列块”划分的场景。
    #   • 三维 grid：再加一个批次/通道维度，适合 “batch × row-tiles × col-tiles” 之类。
    # 本例是对一条一维向量做加法，没有额外的行/列或批次维度要映射，因此仅需一维 grid。
    #
    # Triton 为每个维度提供 program_id(axis=k)。对一维 grid，只取 axis=0 即可；
    # 如果是二维/三维，你会同时取 pid_x = program_id(0)、pid_y = program_id(1)（以及 pid_z）。
    # 这些 id 现在还不是常数，而是由“启动时的 grid 大小”决定的：Triton 会为每个可能的 id 启动一个实例。
    pid = tl.program_id(axis=0)

    # —— 这个program要负责哪一段数据？（把全体索引切成固定大小的分片）———————————————
    # 令 BLOCK_SIZE 为每个实例要处理的元素个数。
    # pid=0 处理 [0..BLOCK_SIZE-1]，
    # pid=1 处理 [BLOCK_SIZE..2*BLOCK_SIZE-1]，以此类推。
    # 举例：n_elements=10、BLOCK_SIZE=4 → 需要 3 个实例：
    #   pid=0 → [0,1,2,3]; 
    #   pid=1 → [4,5,6,7]; 
    #   pid=2 → [8,9]（注意：这片不满 4 个元素）。
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    # 说明：tl.arange(0, BLOCK_SIZE) 生成 0..BLOCK_SIZE-1 这把“刻度尺”，再整体平移到
    # 以 block_start 为起点的那一段，就得到了本实例要访问的全体下标 offsets。
    # 举例：   BLOCK_SIZE=4 时，tl.arange(0, BLOCK_SIZE) 生成 [0,1,2,3]。
    # 如果 pid=1，则 block_start=4，offsets = 4 + [0,1,2,3] = [4,5,6,7]。
    # 如果 pid=2，则 block_start=8，offsets = 8 + [0,1,2,3] = [8,9]。10,11]（后两位越界，见下文）。
    # 这种“连续、等距”的访问能带来更好的内存合并（coalescing），减少显存交易成本。

    # —— 为什么需要 mask？（避免越界与未定义值）———————————————————————
    # 当 n_elements 不是 BLOCK_SIZE 的整数倍时，最后一个实例的部分 offsets 会越过尾端。
    # 直接读这些地址要么触发越界，要么读到未定义值（俗称“垃圾数据”）。
    # mask 精确指出“本次哪些 lane 是有效的”，tl.load/tl.store 会据此跳过无效位置。
    mask = offsets < n_elements

    # —— 把这一段数据搬入寄存器（算术只能在寄存器里进行）———————————————
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)

    # —— 逐元素加法（发生在寄存器里）———————————————————————————————
    output = x + y

    # —— 把结果写回显存；就地写回的说明—————————————————————————————
    # 如果 output_ptr 与 x_ptr 相同，这里等价于“x += y” 的就地更新。
    # 本 kernel 先读后写，且每个实例写入的区间彼此不重叠，因此就地写回在语义与并发上都是安全的。
    tl.store(output_ptr + offsets, output, mask=mask)


在 Triton 中，执行分为两部分：

1. **Device 侧**：如上一段代码所讨论的，由 `@triton.jit` 装饰的函数（如 `add_kernel`），会被编译并在 GPU 上并行运行。  
   它负责实际的数据加载（从显存读入寄存器）、逐元素计算，以及将结果写回显存。  

2. **Host 侧**：写在 Python 里的包装函数（如 `add`），运行在 CPU 上。  
   它的任务是准备输入输出张量、决定并行实例数量（grid），然后“发射” kernel 到 GPU 上执行。  

所以，我们还要声明一个host函数，用于 (1) 分配 `z` 张量
以及 (2) 将上述device侧内核以适当的网格/块大小入队

`add()` 的逻辑可以拆解为三步：  
- **准备输出缓冲区**：分配与输入相同形状和 dtype 的张量；  
- **确定总工作量**：计算要处理的元素个数 (`n_elements`)；  
- **划分并行实例**：根据 `n_elements` 和 `BLOCK_SIZE`，确定 grid 的大小（即要启动多少个 program），再调用 `add_kernel`。

In [ ]:
def add(x: torch.Tensor, y: torch.Tensor):
    
    # 我们需要预先分配输出张量。
    output = torch.empty_like(x)

    # —— 设备一致性检查 ————————————————————————————————————————————————
    assert x.device == DEVICE and y.device == DEVICE and output.device == DEVICE

    # —— 为什么要用 numel()，以及“一维索引区间”的直觉 ————————————————
    # GPU 最终用的是“线性地址”来读写显存：基地址 + 偏移量。
    # 无论你的张量是 [N]、[N,M] 还是 [N,M,K]，在内存里都会按某种顺序线性排开。
    # 因此，最通用、最直接的并行方式是：把“所有元素”看成一条一维的索引带，
    # 索引范围就是 [0, n_elements)。这样每个并行实例只需要知道“我从哪开始、要处理多少个”，
    # 就能通过指针 + 偏移量把正确的数据读出来。
    #
    # numel() 恰好返回“元素的总个数”（各维度长度相乘，与 dtype 无关）——
    # 这就是“一维索引带”的长度。举例：形状 (2,3,4) → numel()=24；
    # 若 BLOCK_SIZE=8，则需要 3 个实例分别处理 [0..7]、[8..15]、[16..23]。
    # 注意：这里默认张量是 contiguous（连续内存）。如果不是，应先 .contiguous()，
    # 否则需要在 kernel 里按 stride 自己做地址计算（本入门例子不展开）。
    n_elements = output.numel()

    # —— grid 的精确定义：lambda meta 与 cdiv 各自解决什么问题 ——————————————
    # 1) 为什么写成 lambda meta？
    #    Triton 在 JIT 编译前会构造一个“元参数字典” meta，并把 *编译期常量*（constexpr）
    #    以及你以关键字传入的元参数（如 BLOCK_SIZE）放进去。把 grid 写成 lambda(meta)
    #    的形式，能保证“grid 的计算”和“kernel 内看到的 BLOCK_SIZE”永远一致。
    #    这在你调参或用 @triton.autotune 探索不同 BLOCK_SIZE 时尤其重要——
    #    Triton 会为每个候选 BLOCK_SIZE 调一次 grid(meta)，得到匹配的并行实例数量。
    #
    # 2) triton.cdiv 是什么？
    #    cdiv(a, b) = ceil(a / b) 的整数实现（等价于 (a + b - 1) // b）。
    #    我们要把 n_elements 个元素按 BLOCK_SIZE 分片，每片交给一个 program；
    #    如果最后一片不满，也要分一个 program（在 kernel 内用 mask 屏蔽越界的 lane）。
    #    因此需要的并行实例数正是 ceil(n_elements / BLOCK_SIZE)。
    #
    # 3) 这如何与 kernel 里的 program_id 对上？
    #    这里的 grid 返回一个一维元组 (Gx,)，其中 Gx=cdiv(... )。
    #    Triton 会并行启动 Gx 个“内核实例”（program），分别带着 pid=0..Gx-1
    #    进入同一份内核代码。于是 kernel 里那一行 pid = tl.program_id(axis=0)
    #    每次运行得到的 pid 都不同——不是固定值；它的取值范围正是由这个 grid 决定的。
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )

    # —— 启动 kernel：同一份 add_kernel 会被为每个 pid 各运行一次 ————————————
    # 张量参数会被隐式转换为设备指针传入；元参数（如 BLOCK_SIZE）必须以关键字形式传入，
    # 这样它才能进入 meta 字典，供上面的 grid(meta) 与 kernel 编译期优化同时使用。
    add_kernel[grid](x, y, output, n_elements, BLOCK_SIZE=1024)

    # —— 异步说明 ————————————————————————————————————————————————
    # kernel 启动是异步的；在需要把结果拿到 CPU 或进行依赖操作时会隐式同步。
    # 若你想手动等待，可以调用 torch.cuda.synchronize()。
    return output


我们现在可以使用上述函数来计算两个 `torch.tensor` 对象的元素和并测试其正确性：

In [9]:
torch.manual_seed(0)
size = 98432
x = torch.rand(size, device=DEVICE)
y = torch.rand(size, device=DEVICE)
output_torch = x + y
output_triton = add(x, y)
print(output_torch)
print(output_triton)
print(f'Torch 的结果和 Triton 的结果之间的最大差异是 '
      f'{torch.max(torch.abs(output_torch - output_triton))}')

tensor([1.3713, 1.3076, 0.4940,  ..., 0.9592, 0.3409, 1.2567], device='cuda:0')
tensor([1.3713, 1.3076, 0.4940,  ..., 0.9592, 0.3409, 1.2567], device='cuda:0')
Torch 的结果和 Triton 的结果之间的最大差异是 0.0


看起来没问题！


## 基准测试

现在，我们可以在规模不断增大的向量上对我们的自定义操作进行基准测试，以了解它相对于 PyTorch 的表现。
为了简化操作，Triton 提供了一组内置实用程序，使我们能够简明地绘制自定义操作的性能图。
针对不同的问题规模。



In [ ]:
@triton.testing.perf_report(
    triton.testing.Benchmark(
        x_names=['size'],  # 用作绘图 x 轴的参数名称
        x_vals=[2**i for i in range(12, 28, 1)],  # `x_name` 的不同取值（输入向量长度）
        x_log=True,  # x 轴使用对数刻度
        line_arg='provider',  # 参数名，不同取值对应图中的不同曲线
        line_vals=['triton', 'torch'],  # `line_arg` 的可能取值
        line_names=['Triton', 'Torch'],  # 曲线的标签名称
        styles=[('blue', '-'), ('green', '-')],  # 曲线的样式
        ylabel='GB/s',  # y 轴标签
        plot_name='vector-add-performance',  # 图的名字（也用作保存文件名）
        args={},  # 对于不在 `x_names` 和 `y_name` 中的函数参数，这里指定它们的取值
    ))
def benchmark(size, provider):
    # —— 输入张量的形状与含义 ———————————————————————————————————————————
    # size 用来决定要生成的张量 x、y 的形状。
    #  - 如果 size 是 int（本教程常用），则 x.shape == y.shape == (size,) —— 一维向量加法的基准。
    #  - 如果 size 是 tuple（例如 (M, N) ），则会生成相同形状的二维张量。但本例的 Triton kernel
    #    是按一维“线性索引”实现的加法，因此通常把 size 设为元素总数（int）来对齐测试。
    # DEVICE 是当前活跃的 PyTorch 设备（通常是 "cuda:0"），dtype 设为 float32。
    x = torch.rand(size, device=DEVICE, dtype=torch.float32)
    y = torch.rand(size, device=DEVICE, dtype=torch.float32)

    # —— quantiles（分位数）用于统计波动 ————————————————————————————————
    # do_bench 会多次运行同一个函数，并返回一组以毫秒计时的统计值。
    # 这里 quantiles = [0.5, 0.2, 0.8] 表示：
    #   - 0.5 分位（中位数，尽量代表“典型”性能）
    #   - 0.2 与 0.8 分位（给出“偏快/偏慢”的范围，观察抖动）
    quantiles = [0.5, 0.2, 0.8]  # 统计性能的分位数

    if provider == 'torch':
        # —— triton.testing.do_bench 是什么？为什么要用 lambda？—————————————
        # triton.testing 是 Triton 提供的测试/基准工具集，do_bench 会：
        #   1) 进行预热（避免首次调用的编译/缓存影响结果）；
        #   2) 在 GPU 上重复执行你提供的“可调用对象”（callable）若干次；
        #   3) 在每次调用前后做必要的同步，保证计时只包含实际内核执行；
        #   4) 返回按 quantiles 统计的运行时间（单位：毫秒）。
        # 这里用 lambda: x + y 的形式创建“无参小函数”，把“要测的那一步”打包起来。
        # lambda 是 Python 的匿名函数写法，作用等同于：
        #   def f(): return x + y
        # 之所以要传 callable，而不是直接写 x + y，是为了让 do_bench 能够多次、可控地重复调用。
        ms, min_ms, max_ms = triton.testing.do_bench(lambda: x + y, quantiles=quantiles)

    if provider == 'triton':
        # —— 测 Triton 版本：调用我们自己写的 add()（内部会发射 Triton kernel）———————
        # 注意：这里的 provider 只是一个字符串标签，用来选择测哪条路径；
        # 它与 Python 模块名 triton 没有关系。
        ms, min_ms, max_ms = triton.testing.do_bench(lambda: add(x, y), quantiles=quantiles)

    # —— 计算内存带宽型吞吐量（GB/s）———————————————————————————————
    # 向量加法的内存交易量 ≈ 读 x + 读 y + 写 output = 3 * numel * element_size bytes。
    #  - x.numel()：元素个数（与维度无关，等于 shape 各维相乘）
    #  - x.element_size()：每个元素占多少字节（float32 为 4）
    #  - ms 是毫秒，需要转成秒（×1e-3）；字节转 GB（×1e-9，使用 10^9 记法）
    # 所以 GB/s = (总字节数 / 10^9) / (时间秒)。
    gbps = lambda ms: 3 * x.numel() * x.element_size() * 1e-9 / (ms * 1e-3)

    # —— 返回三个带宽数值：中位数、慢端、快端 ————————————————————————————————
    # 注意：传入 max_ms（时间更长）得到的是“更低的 GB/s”（悲观端点）；
    #       传入 min_ms（时间更短）得到的是“更高的 GB/s”（乐观端点）。
    # 按当前写法，返回顺序为 (median_gbps, low_gbps, high_gbps)。
    return gbps(ms), gbps(max_ms), gbps(min_ms)


现在我们可以运行上面修饰的函数了。传入 `print_data=True` 可以查看性能数据，传入 `show_plots=True` 可以绘制结果，以及/或者
`save_path='/path/to/results/' 可以将其与原始 CSV 数据一起保存到磁盘：

In [ ]:
benchmark.run(print_data=True, show_plots=True)